In [1]:
import scipy.io as sio
#from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
import tensorflow as tf


#mnist = input_data.read_data_sets('MNIST_data',one_hot=True)


#初始化权值
def weight_variable(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.1)#生成一个截断的正态分布
    return tf.Variable(initial,name=name)

#初始化偏置
def bias_variable(shape,name):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial,name=name)

#卷积层
def conv2d(x,W):
    #x input tensor of shape `[batch, in_height, in_width, in_channels]`
    #W filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    #`strides[0] = strides[3] = 1`. strides[1]代表x方向的步长，strides[2]代表y方向的步长
    #padding: A `string` from: `"SAME", "VALID"`
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层
def max_pool_2x2(x):
    #ksize [1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


#load benchmark dataset
data = sio.loadmat('data\\PDNA-224-PSSM-11.mat')
X = data['data']
Y = data['target']
X = X.reshape(57348,-1)
#loo = LeaveOneOut()
kf = KFold(n_splits=5)
kf.get_n_splits(X)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    #print("len(X_train)={},len(X_test)={}".format(len(X_train),len(X_test)))
    #print("len(Y_train)={},len(Y_test)={}".format(len(Y_train),len(Y_test)))
    print("yes")
    
    N = len(X_train)

    #每个批次的大小
    batch_size = 100
    #计算一共有多少个批次
    n_batch = N // batch_size

    #命名空间
    with tf.name_scope('input'):
        #定义两个placeholder
        x = tf.placeholder(tf.float32,[None,460],name='x-input')
        y = tf.placeholder(tf.float32,[None,2],name='y-input')
        
        with tf.name_scope('x_image'):
            #改变x的格式转为4D的向量[batch, in_height, in_width, in_channels]`
            x_image = tf.reshape(x,[-1,23,20,1],name='x_image')


    with tf.name_scope('Conv1'):
        #初始化第一个卷积层的权值和偏置
        with tf.name_scope('W_conv1'):
            W_conv1 = weight_variable([7,7,1,64],name='W_conv1')#3*3的采样窗口，32个卷积核从1个平面抽取特征
        with tf.name_scope('b_conv1'):  
            b_conv1 = bias_variable([64],name='b_conv1')#每一个卷积核一个偏置值

        #把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
        with tf.name_scope('conv2d_1'):
            conv2d_1 = conv2d(x_image,W_conv1) + b_conv1
        with tf.name_scope('leaky_relu'):
            h_conv1 = tf.nn.leaky_relu(conv2d_1)
        with tf.name_scope('h_pool1'):
            h_pool1 = max_pool_2x2(h_conv1)#进行max-pooling

    with tf.name_scope('Conv2'):
        #初始化第二个卷积层的权值和偏置
        with tf.name_scope('W_conv2'):
            W_conv2 = weight_variable([7,7,64,128],name='W_conv2')#3*3的采样窗口，64个卷积核从32个平面抽取特征
        with tf.name_scope('b_conv2'):  
            b_conv2 = bias_variable([128],name='b_conv2')#每一个卷积核一个偏置值

        #把h_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
        with tf.name_scope('conv2d_2'):
            conv2d_2 = conv2d(h_pool1,W_conv2) + b_conv2
        with tf.name_scope('leaky_relu'):
            h_conv2 = tf.nn.leaky_relu(conv2d_2)
        with tf.name_scope('h_pool2'):
            h_pool2 = max_pool_2x2(h_conv2)#进行max-pooling

    #23*20的图片第一次卷积后还是23*20,第一次池化后变为12*10
    #第二次卷积后为12*10,第二次池化后变为6*5
    #进过上面操作后得到64张6*5的平面

    with tf.name_scope('fc1'):
        #初始化第一个全连接层的权值
        with tf.name_scope('W_fc1'):
            W_fc1 = weight_variable([6*5*256,1024],name='W_fc1')#上一场有6*5*64个神经元，全连接层有512个神经元
        with tf.name_scope('b_fc1'):
            b_fc1 = bias_variable([1024],name='b_fc1')#1024个节点

        #把池化层2的输出扁平化为1维
        with tf.name_scope('h_pool2_flat'):
            h_pool2_flat = tf.reshape(h_pool2,[-1,6*5*256],name='h_pool2_flat')
            
        #求第一个全连接层的输出
        with tf.name_scope('wx_plus_b1'):
            wx_plus_b1 = tf.matmul(h_pool2_flat,W_fc1) + b_fc1
        with tf.name_scope('relu'):
            h_fc1 = tf.nn.relu(wx_plus_b1)


        #keep_prob用来表示神经元的输出概率
        with tf.name_scope('keep_prob'):
            keep_prob = tf.placeholder(tf.float32,name='keep_prob')
        with tf.name_scope('h_fc1_drop'):
            h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob,name='h_fc1_drop')

    with tf.name_scope('fc2'):
        #初始化第二个全连接层
        with tf.name_scope('W_fc2'):
            W_fc2 = weight_variable([1024,2],name='W_fc2')
        with tf.name_scope('b_fc2'):    
            b_fc2 = bias_variable([2],name='b_fc2')
        with tf.name_scope('wx_plus_b2'):
            wx_plus_b2 = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
        with tf.name_scope('softmax'):
            #计算输出
            prediction = tf.nn.softmax(wx_plus_b2)
            #prediction = wx_plus_b2
            
            

    #交叉熵代价函数————————损失函数
    with tf.name_scope('cross_entropy1'):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction),name='cross_entropy1')
    
        

    #使用AdamOptimizer进行优化
    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    #求准确率
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            #结果存放在一个布尔列表中
            correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))#argmax返回一维张量中最大的值所在的位置
            
        with tf.name_scope('accuracy'):
            #求准确率
            accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        
        for epoch in range(20):
            for batch in range(n_batch):
                #训练模型

                if (batch+1)*batch_size > N:
                        batch_xs = X_train[batch*batch_size:N]
                        batch_ys = Y_train[batch*batch_size:N]
                else:
                        batch_xs = X_train[batch*batch_size:(batch+1)*batch_size]
                        batch_ys = Y_train[batch*batch_size:(batch+1)*batch_size]
                        #batch_xs,batch_ys = mnist.train.next_batch(batch_size)
                sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1})


                #print("cross_entropy   ",sess.run(cross_entropy,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1}))

            test_acc = sess.run(accuracy,feed_dict={x:X_test,y:Y_test,keep_prob:1.0})
            #train_acc = sess.run(accuracy,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
            print("Iter: "+ str(epoch) + " , Testing Accuracy=" + str(test_acc)+ " , Training Accuracy=" + "str(train_acc)")



yes
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



InvalidArgumentError: logits and labels must be broadcastable: logits_size=[25,2] labels_size=[50,2]
	 [[Node: cross_entropy1/softmax_cross_entropy_with_logits_sg = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](cross_entropy1/softmax_cross_entropy_with_logits_sg/Reshape, cross_entropy1/softmax_cross_entropy_with_logits_sg/Reshape_1)]]

Caused by op 'cross_entropy1/softmax_cross_entropy_with_logits_sg', defined at:
  File "D:\Program Files\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Program Files\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start

In [ ]:
wx_plus_b2:
 [[-17.76070595  43.17565918]
 [-16.10832024  45.12099457]
 [ -9.37683201  39.95554733]
 [ -9.91409111  39.45625687]
 [ -9.51282787  35.14970779]
 [-11.13184166  35.8319931 ]
 [-16.59625244  41.22007751]
 [-20.83899307  40.71998215]
 [-26.79447746  48.72865295]
 [-24.46959686  44.67155457]]

In [ ]:
import scipy.io as sio
#from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
import tensorflow as tf

data = sio.loadmat('PDNA-224-PSSM-11.mat')
X = data['data']
Y = data['target']
X = X.reshape(57348,-1)
#loo = LeaveOneOut()
kf = KFold(n_splits=5)
kf.get_n_splits(X)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    print("X_train=",X_train[0])
    print('')
    print('')
    print("Y_train=",Y_test[0])
    print('')
   #print("len(X_train)={},len(X_test)={}",Y_train[1],Y_test[1])
    print('')
    print('')
    print('')

In [ ]:
批次为5
yes
Loss: 0.822776
Tensor("cross_entropy_16/cross_entropy:0", shape=(), dtype=float32)
Loss: 0.759299
Tensor("cross_entropy_16/cross_entropy:0", shape=(), dtype=float32)
Loss: 0.778122
Tensor("cross_entropy_16/cross_entropy:0", shape=(), dtype=float32)
Loss: 0.325564
Tensor("cross_entropy_16/cross_entropy:0", shape=(), dtype=float32)
Loss: 0.715323
Tensor("cross_entropy_16/cross_entropy:0", shape=(), dtype=float32)
Loss: 0.640152
Tensor("cross_entropy_16/cross_entropy:0", shape=(), dtype=float32)
Loss: 0.372955

批次为100
yes
Loss: 0.807413
Tensor("cross_entropy_15/cross_entropy:0", shape=(), dtype=float32)
Loss: 0.854778
Tensor("cross_entropy_15/cross_entropy:0", shape=(), dtype=float32)
Loss: 0.804138
Tensor("cross_entropy_15/cross_entropy:0", shape=(), dtype=float32)
Loss: 0.822957
Tensor("cross_entropy_15/cross_entropy:0", shape=(), dtype=float32)
Loss: 0.83096
Tensor("cross_entropy_15/cross_entropy:0", shape=(), dtype=float32)
Loss: 0.836317
Tensor("cross_entropy_15/cross_entropy:0", shape=(), dtype=float32)
Loss: 0.783161

In [ ]:
Iter 0, Testing Accuracy= 0.93034, Training Accuracy=  str(train_acc)
Iter 1, Testing Accuracy= 0.93034, Training Accuracy=  str(train_acc)
Iter 2, Testing Accuracy= 0.93034, Training Accuracy=  str(train_acc)
Iter 3, Testing Accuracy= 0.93034, Training Accuracy=  str(train_acc)
Iter 4, Testing Accuracy= 0.93034, Training Accuracy=  str(train_acc)
yes
Iter 0, Testing Accuracy= 0.920663, Training Accuracy=  str(train_acc)
Iter 1, Testing Accuracy= 0.920663, Training Accuracy=  str(train_acc)
Iter 2, Testing Accuracy= 0.920663, Training Accuracy=  str(train_acc)
Iter 3, Testing Accuracy= 0.920663, Training Accuracy=  str(train_acc)
Iter 4, Testing Accuracy= 0.920663, Training Accuracy=  str(train_acc)
yes
Iter 0, Testing Accuracy= 0.939146, Training Accuracy=  str(train_acc)
Iter 1, Testing Accuracy= 0.939146, Training Accuracy=  str(train_acc)
Iter 2, Testing Accuracy= 0.939146, Training Accuracy=  str(train_acc)
Iter 3, Testing Accuracy= 0.939146, Training Accuracy=  str(train_acc)
Iter 4, Testing Accuracy= 0.939146, Training Accuracy=  str(train_acc)
yes
Iter 0, Testing Accuracy= 0.94193, Training Accuracy=  str(train_acc)
Iter 1, Testing Accuracy= 0.94193, Training Accuracy=  str(train_acc)
Iter 2, Testing Accuracy= 0.94193, Training Accuracy=  str(train_acc)
Iter 3, Testing Accuracy= 0.94193, Training Accuracy=  str(train_acc)
Iter 4, Testing Accuracy= 0.94193, Training Accuracy=  str(train_acc)
yes
Iter 0, Testing Accuracy= 0.93853, Training Accuracy=  str(train_acc)
Iter 1, Testing Accuracy= 0.93853, Training Accuracy=  str(train_acc)
Iter 2, Testing Accuracy= 0.93853, Training Accuracy=  str(train_acc)
Iter 3, Testing Accuracy= 0.93853, Training Accuracy=  str(train_acc)
Iter 4, Testing Accuracy= 0.93853, Training Accuracy=  str(train_acc)